In [1]:
import websockets as ws
import json
from datetime import datetime
import time
import math


In [2]:
async def send_boot_notification(websocket):
        
        print("IM BOOT")
        msg = [2, "0jdsEnnyo2kpCP8FLfHlNpbvQXosR5ZNlh8v", "BootNotification", {
            "chargePointVendor": "AVT-Company",
            "chargePointModel": "AVT-Express",
            "chargePointSerialNumber": "avt.001.13.1",
            "chargeBoxSerialNumber": "avt.001.13.1.01",
            "firmwareVersion": "0.9.87",
            "iccid": "",
            "imsi": "",
            "meterType": "AVT NQC-ACDC",
            "meterSerialNumber": "avt.001.13.1.01"}]
        msg_send = json.dumps(msg)
        await websocket.send(msg_send)
        print("SENT")

In [3]:
from datetime import datetime


async def send_data_transfer_request(websocket, message):
    dt = datetime.now()
    conf_msg = [3,
                    message[1],
                    "DataTransfer",
                    {"status": "Accepted"}]
    conf_send = json.dumps(conf_msg)
    print("Sending confirmation: " + conf_send)
    try:
        await websocket.send(conf_send)
        print("Message went away")
    except Exception as e:
        print(str(e))

In [4]:
async def listen_for_response(websocket):
    while True:
        try:
            json_formatted_message = await websocket.recv()
            message = json.loads(json_formatted_message)
            print("Got message:")
            print(message)
            if message[2] == "DataTransfer":
                await send_data_transfer_request(websocket,message)
        except Exception as e:
            print(str(e))
    

In [5]:
try:
    async with ws.connect(
        'ws://18.202.253.30:1337/charger/testnumber13/',
        subprotocols=['ocpp1.6'],
        ping_interval=5,
        timeout = None
        ) as ws:
            websocket = ws
            print("success")
            await send_boot_notification(websocket)
            await listen_for_response(websocket)
except Exception as e:
    print("fail")
    print(str(e))

success
IM BOOT
fail
no close frame received or sent
